In [12]:
# -*- coding: utf-8 -*-
"""
PepMLM_Peptide_Generation_Example.ipynb

### ESM-2 모델을 사용한 타겟-특이적 펩타이드 서열 생성 예제

이 노트북은 Hugging Face의 `transformers` 라이브러리를 사용하여,
특정 타겟 단백질 서열을 조건으로 주어 새로운 펩타이드 서열을 생성하는 방법을 보여줍니다.
[최종 수정] 생성 방식의 한계를 해결하기 위해, 모델의 본래 목적인 '마스크 채우기' 방식으로 전환하여 안정적인 결과를 생성합니다.
"""

# 1. 필수 라이브러리 설치
# transformers와 torch 라이브러리를 설치합니다.
!pip install transformers torch

# 2. 라이브러리 임포트
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM

# 3. 모델 및 토크나이저 로드
# 안정적인 표준 모델인 facebook/esm2_t12_35M_UR50D를 사용합니다.
model_name = "facebook/esm2_t12_35M_UR50D"
print(f"'{model_name}' 모델과 토크나이저를 로딩합니다...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    print("모델과 토크나이저 로딩 완료!")
except Exception as e:
    print(f"모델 로딩 중 오류가 발생했습니다: {e}")
    raise e

# 4. 타겟 단백질 서열 및 생성 파라미터 설정
target_protein_sequence = "PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK"
max_peptide_length = 10
num_candidates = 5
temperature = 1.0 # 생성의 무작위성을 조절
top_k = 50       # 확률이 높은 상위 K개 토큰 중에서만 샘플링

print("\n--- 생성 정보 ---")
print(f"타겟 단백질 서열: {target_protein_sequence[:30]}...")
print(f"생성할 펩타이드 최대 길이: {max_peptide_length}")
print(f"생성할 후보군 개수: {num_candidates}")
print("--------------------")

# 5. 모델 입력용 프롬프트 생성
# [수정됨] "빈칸 채우기" 방식으로 프롬프트를 구성합니다.
# [타겟 서열] [MASK] [MASK]... 형태로 모델에게 명확한 과제를 제시합니다.
formatted_target = " ".join(list(target_protein_sequence))
mask_tokens = " ".join([tokenizer.mask_token] * max_peptide_length)

prompt = f"{tokenizer.cls_token} {formatted_target} {tokenizer.eos_token} {mask_tokens}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# 6. 펩타이드 서열 생성 실행 (반복적 마스크 채우기)
print("\n펩타이드 서열 생성을 시작합니다 (반복적 마스크 채우기 방식)...")

# MASK 토큰의 위치를 미리 찾아둡니다.
mask_token_indices = (input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

generated_sequences = []
with torch.no_grad():
    for i in range(num_candidates):
        # 각 후보마다 원본 MASK 프롬프트에서 시작합니다.
        current_ids = input_ids.clone()

        # 다양성을 위해 마스크를 채우는 순서를 무작위로 섞습니다.
        shuffled_mask_indices = mask_token_indices[torch.randperm(len(mask_token_indices))]

        for mask_idx in shuffled_mask_indices:
            # 모델을 통해 로짓(확률)을 얻습니다.
            outputs = model(input_ids=current_ids)
            logits = outputs.logits

            # 현재 채우려는 MASK 위치의 로짓만 추출합니다.
            mask_logits = logits[0, mask_idx, :]

            # 샘플링을 위한 후처리 (Temperature, Top-K)
            mask_logits = mask_logits / temperature
            effective_top_k = min(top_k, tokenizer.vocab_size)
            top_k_values, top_k_indices = torch.topk(mask_logits, effective_top_k)
            filter_tensor = torch.full_like(mask_logits, -float('Inf'))
            filter_tensor.scatter_(0, top_k_indices, top_k_values)

            # 확률 분포로 변환 후 샘플링
            probs = F.softmax(filter_tensor, dim=-1)
            predicted_token_id = torch.multinomial(probs, num_samples=1)

            # MASK 토큰을 예측된 아미노산 토큰으로 교체합니다.
            current_ids[0, mask_idx] = predicted_token_id.item()

        generated_sequences.append(current_ids)

print("생성 완료!")

# 7. 생성 결과 디코딩 및 출력
print(f"\n--- 생성된 펩타이드 후보 (상위 {num_candidates}개) ---")

for i, sequence_tensor in enumerate(generated_sequences):
    # 최종 텐서에서 원래 MASK가 있던 위치의 토큰들만 추출합니다.
    generated_token_ids = sequence_tensor[0, mask_token_indices]

    # 추출된 토큰들을 디코딩하여 펩타이드 서열로 변환합니다.
    peptide_part = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

    # 아미노산 사이의 공백을 제거합니다.
    peptide_part_no_space = "".join(peptide_part.split())

    print(f"후보 {i+1}: {peptide_part_no_space} (길이: {len(peptide_part_no_space)})")

print("\n--- 예제 실행 완료 ---")


'facebook/esm2_t12_35M_UR50D' 모델과 토크나이저를 로딩합니다...
모델과 토크나이저 로딩 완료!

--- 생성 정보 ---
타겟 단백질 서열: PIAQIHILEGRSDEQKETLIREVSEAISRS...
생성할 펩타이드 최대 길이: 10
생성할 후보군 개수: 5
--------------------

펩타이드 서열 생성을 시작합니다 (반복적 마스크 채우기 방식)...
생성 완료!

--- 생성된 펩타이드 후보 (상위 5개) ---
후보 1: PQAAQRHGGL (길이: 10)
후보 2: RYSPEDSEKQ (길이: 10)
후보 3: AGELPADTSS (길이: 10)
후보 4: TYNKDDKKLT (길이: 10)
후보 5: NRCTWEYLSM (길이: 10)

--- 예제 실행 완료 ---
